# Keras Tutorial: Artificial Neural Networks (ANNs)

## Welcome to Keras!

This comprehensive tutorial will teach you how to build, train, and evaluate Artificial Neural Networks using **Keras 3** with TensorFlow backend.

### Learning Objectives

By the end of this notebook, you will be able to:
- Build neural network models using Sequential and Functional APIs
- Understand and use different layers, activations, and optimizers
- Train models for classification and regression tasks
- Evaluate model performance and make predictions
- Apply regularization techniques
- Save and load trained models

### Prerequisites
- Basic Python knowledge
- Understanding of basic machine learning concepts
- NumPy fundamentals (helpful)

Let's get started! 🚀

---
## Section 1: Introduction & Setup

First, let's import the necessary libraries and check that everything is installed correctly.

In [1]:
# Import core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Core libraries imported successfully!")

Core libraries imported successfully!


In [2]:
# Import TensorFlow and Keras
import tensorflow as tf
import keras
from keras import layers, models, optimizers, losses, metrics

# Check versions
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"\nKeras backend: {keras.backend.backend()}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("\n✓ Keras is ready to use!")

TensorFlow version: 2.20.0
Keras version: 3.13.0

Keras backend: tensorflow

✓ Keras is ready to use!


### Understanding Keras 3

**Keras 3** is a multi-backend deep learning framework that can run on:
- **TensorFlow** (default, what we're using)
- **JAX** (for high-performance computing)
- **PyTorch** (for PyTorch enthusiasts)

This makes Keras extremely flexible! For this tutorial, we're using the TensorFlow backend, which is the most popular and well-supported option.

**Key advantages of Keras:**
- 🎯 User-friendly API - easy to learn
- 🏗️ Modular design - build complex models easily  
- 📚 Excellent documentation
- 🚀 Production-ready with TensorFlow ecosystem

---
## Section 2: Data Fundamentals

Before building models, we need to understand how to work with data in Keras.

In [ ]:
# Load a sample dataset - MNIST (handwritten digits)
from keras.datasets import mnist

# Load data - returns train and test splits
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("Dataset loaded successfully!")
print(f"\nTraining samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Image shape: {X_train.shape[1:]}")
print(f"Number of classes: {len(np.unique(y_train))}")

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Sample MNIST Digits', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(f'Label: {y_train[i]}')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Each image is a 28x28 pixel grayscale image of a handwritten digit (0-9)")

### Data Preprocessing

Neural networks work best when:
1. **Data is normalized** (scaled to 0-1 or -1 to 1)
2. **Data is properly shaped** (flattened for ANNs)
3. **Labels are in the correct format** (one-hot encoded for multi-class)

Let's prepare our data:

In [ ]:
# Step 1: Flatten images from 28x28 to 784 (for ANN input)
X_train_flat = X_train.reshape(X_train.shape[0], -1)  # -1 means "infer dimension"
X_test_flat = X_test.reshape(X_test.shape[0], -1)

print(f"Original shape: {X_train.shape}")
print(f"Flattened shape: {X_train_flat.shape}")
print(f"\nEach image is now a vector of {X_train_flat.shape[1]} pixels")

In [ ]:
# Step 2: Normalize pixel values from [0, 255] to [0, 1]
X_train_norm = X_train_flat.astype('float32') / 255.0
X_test_norm = X_test_flat.astype('float32') / 255.0

print(f"Original pixel range: [{X_train_flat.min()}, {X_train_flat.max()}]")
print(f"Normalized pixel range: [{X_train_norm.min():.2f}, {X_train_norm.max():.2f}]")
print("\n✓ Data normalized successfully!")

In [ ]:
# Step 3: Convert labels to categorical (one-hot encoding)
from keras.utils import to_categorical

# Before: labels are integers (0-9)
print(f"Original labels: {y_train[:5]}")

# After: labels are one-hot vectors
y_train_cat = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

print(f"\nOne-hot encoded label for '{y_train[0]}':")
print(y_train_cat[0])
print(f"\nLabel shape: {y_train_cat.shape}")

**Why one-hot encoding?**  
For multi-class classification, we convert integer labels (like 5) to vectors like `[0,0,0,0,0,1,0,0,0,0]`. This helps the network learn that classes are distinct categories, not ordered numbers.

---
## Section 3: Building Your First Model - Sequential API

Now let's build our first neural network! We'll use the **Sequential API**, which is perfect for models where data flows linearly through layers.

### Understanding Neural Network Architecture

An Artificial Neural Network (ANN) consists of:
- **Input Layer**: Receives the data (784 neurons for our flattened 28x28 images)
- **Hidden Layers**: Process the data (we choose how many neurons)
- **Output Layer**: Produces predictions (10 neurons for 10 digit classes)

Let's build a simple model:

In [ ]:
# Create a Sequential model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),  # Hidden layer 1
    layers.Dense(64, activation='relu'),                        # Hidden layer 2  
    layers.Dense(10, activation='softmax')                      # Output layer
])

print("Model created successfully!")
print("\nLet's examine the architecture:")

In [ ]:
# View model architecture
model.summary()

print("\n💡 Key observations:")
print("- Total parameters: ~100K (these are learned during training)")
print("- Most parameters are in the first layer (784 inputs × 128 neurons)")
print("- Output layer has 10 neurons (one for each digit class)")

### Understanding the Layers

Let's break down what each layer does:

1. **`Dense(128, activation='relu', input_shape=(784,))`**
   - `Dense`: Fully connected layer (every input connects to every output)
   - `128`: Number of neurons in this layer
   - `activation='relu'`: ReLU activation function (more on this later)
   - `input_shape=(784,)`: Expected input dimension (only needed for first layer)

2. **`Dense(64, activation='relu')`**
   - Second hidden layer with 64 neurons
   - No input_shape needed (Keras infers it from previous layer)

3. **`Dense(10, activation='softmax')`**
   - Output layer with 10 neurons (one per class)
   - `softmax`: Converts outputs to probabilities that sum to 1

### Project 1: Binary Classification with Iris Dataset

Let's apply what we learned to a simpler problem: binary classification (2 classes).

In [ ]:
# Load Iris dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
iris = load_iris()
X = iris.data
y = iris.target

# Binary classification: Setosa (0) vs. Not Setosa (1 or 2)
y_binary = (y != 0).astype(int)

print(f"Total samples: {len(X)}")
print(f"Features: {iris.feature_names}")
print(f"\nClass distribution:")
print(f"  Setosa: {np.sum(y_binary == 0)}")
print(f"  Not Setosa: {np.sum(y_binary == 1)}")

In [ ]:
# Split data
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X, y_binary, test_size=0.2, random_state=42
)

# Normalize features (important for neural networks!)
scaler = StandardScaler()
X_train_iris = scaler.fit_transform(X_train_iris)
X_test_iris = scaler.transform(X_test_iris)

print(f"Training samples: {len(X_train_iris)}")
print(f"Test samples: {len(X_test_iris)}")
print("\n✓ Data prepared for binary classification")

In [ ]:
# Build binary classification model
binary_model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(4,)),  # 4 input features
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # sigmoid for binary classification
], name='binary_classifier')

binary_model.summary()

print("\n💡 Note: For binary classification, we use:")
print("  - 1 output neuron (instead of 2)")
print("  - sigmoid activation (outputs probability between 0 and 1)")

---
## Section 4: Model Compilation

Before training, we must **compile** the model by specifying:
1. **Optimizer**: How the model updates its weights
2. **Loss function**: What the model tries to minimize
3. **Metrics**: What we use to evaluate performance

### Understanding Optimizers

**Common optimizers:**
- **SGD**: Stochastic Gradient Descent (classic, requires careful tuning)
- **Adam**: Adaptive Moment Estimation (most popular, works well out-of-the-box)
- **RMSprop**: Root Mean Square Propagation (good for RNNs)

**For beginners, use Adam** - it's fast and reliable!

In [ ]:
# Compile the binary classification model
binary_model.compile(
    optimizer='adam',                    # Adam optimizer with default learning rate (0.001)
    loss='binary_crossentropy',          # Loss for binary classification
    metrics=['accuracy']                 # Track accuracy during training
)

print("✓ Model compiled successfully!")
print("\nCompilation settings:")
print(f"  Optimizer: Adam")
print(f"  Loss: Binary Crossentropy")
print(f"  Metrics: Accuracy")

### Loss Functions Guide

Choose your loss function based on the problem:

| Problem Type | Output Activation | Loss Function |
|--------------|-------------------|---------------|
| Binary Classification | `sigmoid` | `binary_crossentropy` |
| Multi-class Classification | `softmax` | `categorical_crossentropy` |
| Multi-class (integer labels) | `softmax` | `sparse_categorical_crossentropy` |
| Regression | None or `linear` | `mean_squared_error` or `mae` |

In [ ]:
# Example: Compile with different optimizers and learning rates

# Option 1: Adam with custom learning rate
# binary_model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=0.01),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

# Option 2: SGD with momentum
# binary_model.compile(
#     optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

print("💡 You can experiment with different optimizers and hyperparameters!")

---
## Section 5: Training Models

Now comes the exciting part - training our model! We use the `fit()` method.

In [ ]:
# Train the binary classification model
history = binary_model.fit(
    X_train_iris,           # Training data
    y_train_iris,           # Training labels
    epochs=50,              # Number of times to iterate over the entire dataset
    batch_size=16,          # Number of samples per gradient update
    validation_split=0.2,   # Use 20% of training data for validation
    verbose=1               # Print progress (0=silent, 1=progress bar, 2=one line per epoch)
)

print("\n✓ Training completed!")

### Understanding Training Parameters

- **`epochs`**: How many times the model sees the entire dataset
  - More epochs = more learning (but risk of overfitting)
  - Start with 10-50 epochs

- **`batch_size`**: How many samples to process before updating weights
  - Smaller batches (16-32): More updates, better generalization
  - Larger batches (128-256): Faster training, more stable

- **`validation_split`**: Percentage of training data to use for validation
  - Helps monitor overfitting
  - Typical values: 0.1 to 0.2

In [ ]:
# Visualize training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss
ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Model Loss Over Time', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot accuracy
ax2.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Model Accuracy Over Time', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Good signs:")
print("  - Loss decreases over time")
print("  - Training and validation curves are close (no overfitting)")
print("  - Accuracy increases over time")

In [ ]:
# Alternative: Using validation_data instead of validation_split
# This gives you more control over the validation set

# history = binary_model.fit(
#     X_train_iris,
#     y_train_iris,
#     epochs=50,
#     batch_size=16,
#     validation_data=(X_test_iris, y_test_iris)  # Use test set for validation
# )

print("💡 You can also pass validation_data=(X_val, y_val) explicitly")

---
## Section 6: Model Evaluation & Prediction

After training, we need to evaluate how well our model performs on unseen data.

In [ ]:
# Evaluate on test data
test_loss, test_accuracy = binary_model.evaluate(X_test_iris, y_test_iris, verbose=0)

print("📊 Test Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

print("\n✓ Model evaluation complete!")

In [ ]:
# Make predictions
predictions = binary_model.predict(X_test_iris, verbose=0)

print("Sample predictions (probabilities):")
print(predictions[:5].flatten())

# Convert probabilities to class labels (0 or 1)
predicted_classes = (predictions > 0.5).astype(int).flatten()

print("\nPredicted classes:")
print(predicted_classes[:10])

print("\nActual classes:")
print(y_test_iris[:10])

In [ ]:
# Confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
cm = confusion_matrix(y_test_iris, predicted_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.show()

# Detailed metrics
print("\nClassification Report:")
print(classification_report(y_test_iris, predicted_classes, 
                          target_names=['Setosa', 'Not Setosa']))

### Understanding Predictions

For **binary classification**:
- `model.predict()` returns probabilities (values between 0 and 1)
- We convert to class labels using a threshold (typically 0.5)
- Probability > 0.5 → Class 1, Probability ≤ 0.5 → Class 0

For **multi-class classification** (coming up next):
- `model.predict()` returns probability for each class
- We use `argmax()` to get the class with highest probability

In [ ]:
# Example: Making predictions on new data
new_sample = X_test_iris[0:1]  # Take first test sample

prediction_prob = binary_model.predict(new_sample, verbose=0)[0][0]
prediction_class = int(prediction_prob > 0.5)

print(f"New sample features: {new_sample[0]}")
print(f"\nPredicted probability: {prediction_prob:.4f}")
print(f"Predicted class: {prediction_class} ({'Not Setosa' if prediction_class else 'Setosa'})")
print(f"Actual class: {y_test_iris[0]} ({'Not Setosa' if y_test_iris[0] else 'Setosa'})")

---
## Section 7: Functional API

The **Functional API** is more flexible than Sequential API. It's useful for:
- Models with multiple inputs/outputs
- Models with shared layers
- Non-linear topologies (e.g., residual connections)

Let's rebuild our binary classifier using Functional API:

In [ ]:
# Functional API approach
from keras import Input, Model

# Define input layer
inputs = Input(shape=(4,), name='input_layer')

# Build architecture by calling layers on previous layer
x = layers.Dense(16, activation='relu', name='hidden_1')(inputs)
x = layers.Dense(8, activation='relu', name='hidden_2')(x)
outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

# Create model
functional_model = Model(inputs=inputs, outputs=outputs, name='functional_binary_classifier')

functional_model.summary()

print("\n✓ Functional model created!")
print("\n💡 Notice: Same architecture, different syntax")

### Sequential vs Functional API

**Sequential API:**
```python
model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(4,)),
    layers.Dense(1, activation='sigmoid')
])
```

**Functional API:**
```python
inputs = Input(shape=(4,))
x = layers.Dense(16, activation='relu')(inputs)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)
```

**When to use which?**
- Use **Sequential** for simple, linear models (most cases)
- Use **Functional** for complex architectures

In [ ]:
# Compile and train the functional model
functional_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train (quietly this time)
history_func = functional_model.fit(
    X_train_iris, y_train_iris,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    verbose=0
)

# Evaluate
test_loss, test_acc = functional_model.evaluate(X_test_iris, y_test_iris, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print("\n✓ Functional model trained successfully!")

### Example: Multi-Input Model (Advanced Preview)

Here's a preview of what Functional API enables (you don't need to run this now):

In [ ]:
# Example of multi-input model (for illustration only)

# Two separate inputs
input_1 = Input(shape=(10,), name='input_1')
input_2 = Input(shape=(5,), name='input_2')

# Process each input separately
x1 = layers.Dense(32, activation='relu')(input_1)
x2 = layers.Dense(16, activation='relu')(input_2)

# Merge the two paths
merged = layers.concatenate([x1, x2])

# Final output
output = layers.Dense(1, activation='sigmoid')(merged)

# Create multi-input model
multi_input_model = Model(inputs=[input_1, input_2], outputs=output)

print("Multi-input model architecture:")
multi_input_model.summary()

print("\n💡 This is just a preview - Sequential API can't do this!")

---
## Section 8: Common Layers & Techniques

Let's explore important layers and techniques for building robust neural networks.

### Activation Functions

Activation functions introduce **non-linearity** to the network, allowing it to learn complex patterns.

**Common activations:**
- **ReLU** (Rectified Linear Unit): `max(0, x)` - most popular for hidden layers
- **Sigmoid**: Outputs between 0 and 1 - used for binary classification
- **Tanh**: Outputs between -1 and 1 - alternative to ReLU
- **Softmax**: Converts to probabilities - used for multi-class classification

In [ ]:
# Visualize activation functions
x = np.linspace(-5, 5, 100)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# ReLU
axes[0, 0].plot(x, np.maximum(0, x), linewidth=2)
axes[0, 0].set_title('ReLU: max(0, x)', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)
axes[0, 0].axvline(x=0, color='k', linewidth=0.5)

# Sigmoid
axes[0, 1].plot(x, 1 / (1 + np.exp(-x)), linewidth=2)
axes[0, 1].set_title('Sigmoid: 1 / (1 + e^(-x))', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0.5, color='r', linewidth=0.5, linestyle='--')

# Tanh
axes[1, 0].plot(x, np.tanh(x), linewidth=2)
axes[1, 0].set_title('Tanh: (e^x - e^(-x)) / (e^x + e^(-x))', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)

# Linear
axes[1, 1].plot(x, x, linewidth=2)
axes[1, 1].set_title('Linear: f(x) = x', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=0, color='k', linewidth=0.5)
axes[1, 1].axvline(x=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print("💡 Choose activation based on layer:")
print("  - Hidden layers: ReLU (default choice)")
print("  - Binary output: Sigmoid")
print("  - Multi-class output: Softmax")
print("  - Regression output: Linear (or none)")

### Dropout - Regularization Technique

**Dropout** randomly "drops" (sets to 0) a percentage of neurons during training. This:
- Prevents overfitting
- Makes the network more robust
- Acts like training an ensemble of networks

Typical dropout rate: 0.2 to 0.5

In [ ]:
# Model with Dropout
model_with_dropout = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dropout(0.3),  # Drop 30% of neurons randomly
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    
    layers.Dense(10, activation='softmax')
], name='model_with_dropout')

model_with_dropout.summary()

print("\n💡 Dropout layers have no trainable parameters")
print("   They only activate during training, not during inference")

### Batch Normalization

**Batch Normalization** normalizes the inputs of each layer. Benefits:
- Faster training
- Higher learning rates possible
- Less sensitive to initialization
- Acts as regularization

Place it after Dense layers (before or after activation - both work)

In [ ]:
# Model with Batch Normalization
model_with_bn = keras.Sequential([
    layers.Dense(128, input_shape=(784,)),
    layers.BatchNormalization(),  # Normalize outputs
    layers.Activation('relu'),
    
    layers.Dense(64),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(10, activation='softmax')
], name='model_with_batchnorm')

model_with_bn.summary()

print("\n💡 BatchNormalization has trainable parameters (gamma and beta)")
print("   It learns the optimal normalization for each layer")

### Combining Techniques

You can combine multiple techniques for better performance:

In [ ]:
# Advanced model with multiple techniques
advanced_model = keras.Sequential([
    layers.Dense(128, input_shape=(784,)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    
    layers.Dense(64),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    
    layers.Dense(10, activation='softmax')
], name='advanced_model')

advanced_model.summary()

print("\n✓ This model combines:")
print("  - Batch Normalization (faster training, regularization)")
print("  - Dropout (prevent overfitting)")
print("  - ReLU activation (non-linearity)")

---
## Section 9: Complete Project - Multi-Class Classification

Let's put everything together in a complete MNIST digit classification project!

In [ ]:
# We already loaded and preprocessed MNIST earlier
print("Data shapes:")
print(f"  X_train: {X_train_norm.shape}")
print(f"  y_train: {y_train_cat.shape}")
print(f"  X_test: {X_test_norm.shape}")
print(f"  y_test: {y_test_cat.shape}")
print("\n✓ Data ready for multi-class classification!")

In [ ]:
# Build the final MNIST classifier
mnist_model = keras.Sequential([
    # Input layer + first hidden layer
    layers.Dense(256, input_shape=(784,)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    
    # Second hidden layer
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    
    # Third hidden layer
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    
    # Output layer
    layers.Dense(10, activation='softmax')
], name='mnist_classifier')

mnist_model.summary()

In [ ]:
# Compile the model
mnist_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✓ Model compiled and ready for training!")

In [ ]:
# Train the model
history_mnist = mnist_model.fit(
    X_train_norm,
    y_train_cat,
    epochs=20,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

print("\n✓ Training completed!")

In [ ]:
# Visualize training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history_mnist.history['loss'], label='Training', linewidth=2)
ax1.plot(history_mnist.history['val_loss'], label='Validation', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('MNIST Model Loss', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy
ax2.plot(history_mnist.history['accuracy'], label='Training', linewidth=2)
ax2.plot(history_mnist.history['val_accuracy'], label='Validation', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('MNIST Model Accuracy', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = mnist_model.evaluate(X_test_norm, y_test_cat, verbose=0)

print("📊 Final Test Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

In [ ]:
# Make predictions
predictions_mnist = mnist_model.predict(X_test_norm, verbose=0)

# Convert from one-hot to class labels
predicted_classes = np.argmax(predictions_mnist, axis=1)
true_classes = np.argmax(y_test_cat, axis=1)

print("Sample predictions:")
print(f"Predicted: {predicted_classes[:10]}")
print(f"Actual:    {true_classes[:10]}")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
fig.suptitle('MNIST Predictions', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i], cmap='gray')
    
    # Color based on correctness
    color = 'green' if predicted_classes[i] == true_classes[i] else 'red'
    confidence = predictions_mnist[i][predicted_classes[i]] * 100
    
    ax.set_title(f'Pred: {predicted_classes[i]} ({confidence:.1f}%)\nTrue: {true_classes[i]}',
                color=color, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Green = Correct, Red = Incorrect")

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.title('MNIST Confusion Matrix', fontsize=14, fontweight='bold')
plt.show()

print("💡 Diagonal values are correct predictions")
print("   Off-diagonal values are errors")

### Analyzing Errors

In [ ]:
# Find misclassified examples
misclassified_idx = np.where(predicted_classes != true_classes)[0]

print(f"Total misclassified: {len(misclassified_idx)} / {len(true_classes)}")
print(f"Error rate: {len(misclassified_idx) / len(true_classes) * 100:.2f}%")

# Show some errors
if len(misclassified_idx) > 0:
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle('Misclassified Examples', fontsize=16, fontweight='bold', color='red')
    
    for i, ax in enumerate(axes.flat):
        if i < len(misclassified_idx):
            idx = misclassified_idx[i]
            ax.imshow(X_test[idx], cmap='gray')
            confidence = predictions_mnist[idx][predicted_classes[idx]] * 100
            ax.set_title(f'Predicted: {predicted_classes[idx]} ({confidence:.1f}%)\nActual: {true_classes[idx]}',
                        color='red', fontweight='bold')
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

---
## Section 10: Regression with ANNs

Now let's tackle a different type of problem: **regression** (predicting continuous values).

In [ ]:
# Load California Housing dataset
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X_housing = housing.data
y_housing = housing.target

print("California Housing Dataset")
print(f"Samples: {X_housing.shape[0]}")
print(f"Features: {X_housing.shape[1]}")
print(f"\nFeature names: {housing.feature_names}")
print(f"\nTarget: Median house value (in $100,000s)")
print(f"Target range: ${y_housing.min()*100000:.0f} - ${y_housing.max()*100000:.0f}")

In [ ]:
# Visualize target distribution
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(y_housing, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('House Value ($100k)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of House Prices', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(y_housing, vert=True)
plt.ylabel('House Value ($100k)', fontsize=12)
plt.title('House Price Box Plot', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Split and normalize data
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

# Normalize features (CRITICAL for regression!)
scaler_housing = StandardScaler()
X_train_h = scaler_housing.fit_transform(X_train_h)
X_test_h = scaler_housing.transform(X_test_h)

print(f"Training samples: {len(X_train_h)}")
print(f"Test samples: {len(X_test_h)}")
print("\n✓ Data prepared for regression")

### Key Differences for Regression

Compared to classification:

1. **Output layer**: 1 neuron (instead of multiple classes)
2. **Output activation**: None or linear (instead of softmax/sigmoid)
3. **Loss function**: MSE or MAE (instead of crossentropy)
4. **Metrics**: MAE, MSE, R² (instead of accuracy)

In [ ]:
# Build regression model
regression_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(8,)),
    layers.Dropout(0.2),
    
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    
    layers.Dense(16, activation='relu'),
    
    # Output layer for regression
    layers.Dense(1)  # No activation (linear output)
], name='housing_price_predictor')

regression_model.summary()

In [ ]:
# Compile with regression-specific settings
regression_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',  # Mean Squared Error
    metrics=['mae']  # Mean Absolute Error
)

print("✓ Regression model compiled!")
print("\nRegression settings:")
print("  Loss: MSE (penalizes large errors more)")
print("  Metric: MAE (average absolute error in $100k)")

In [ ]:
# Train the regression model
history_reg = regression_model.fit(
    X_train_h,
    y_train_h,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

print("✓ Training completed!")

In [ ]:
# Visualize training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# MSE
ax1.plot(history_reg.history['loss'], label='Training MSE', linewidth=2)
ax1.plot(history_reg.history['val_loss'], label='Validation MSE', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('MSE', fontsize=12)
ax1.set_title('Mean Squared Error', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MAE
ax2.plot(history_reg.history['mae'], label='Training MAE', linewidth=2)
ax2.plot(history_reg.history['val_mae'], label='Validation MAE', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('MAE', fontsize=12)
ax2.set_title('Mean Absolute Error', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
test_mse, test_mae = regression_model.evaluate(X_test_h, y_test_h, verbose=0)

print("📊 Test Results:")
print(f"  MSE: {test_mse:.4f}")
print(f"  MAE: {test_mae:.4f} (average error of ${test_mae*100000:.0f})")
print(f"  RMSE: {np.sqrt(test_mse):.4f}")

In [ ]:
# Make predictions
predictions_reg = regression_model.predict(X_test_h, verbose=0).flatten()

# Calculate R² score
from sklearn.metrics import r2_score
r2 = r2_score(y_test_h, predictions_reg)

print(f"R² Score: {r2:.4f}")
print(f"\n💡 R² = {r2:.2%} of variance explained by the model")

In [ ]:
# Visualize predictions vs actual
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot
ax1.scatter(y_test_h, predictions_reg, alpha=0.5, s=20)
ax1.plot([y_test_h.min(), y_test_h.max()], 
         [y_test_h.min(), y_test_h.max()], 
         'r--', linewidth=2, label='Perfect predictions')
ax1.set_xlabel('Actual Price ($100k)', fontsize=12)
ax1.set_ylabel('Predicted Price ($100k)', fontsize=12)
ax1.set_title(f'Predictions vs Actual (R²={r2:.3f})', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Residuals
residuals = y_test_h - predictions_reg
ax2.scatter(predictions_reg, residuals, alpha=0.5, s=20)
ax2.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax2.set_xlabel('Predicted Price ($100k)', fontsize=12)
ax2.set_ylabel('Residuals', fontsize=12)
ax2.set_title('Residual Plot', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Good model: Points close to red line, residuals centered around 0")

In [ ]:
# Sample predictions
print("Sample Predictions:")
print("\nActual | Predicted | Error")
print("-" * 35)
for i in range(10):
    actual = y_test_h[i] * 100000
    pred = predictions_reg[i] * 100000
    error = abs(actual - pred)
    print(f"${actual:,.0f} | ${pred:,.0f} | ${error:,.0f}")

---
## Section 11: Saving & Loading Models

After training a model, you'll want to save it for later use.

In [ ]:
# Save model in Keras format (recommended)
mnist_model.save('mnist_model.keras')
print("✓ Model saved as 'mnist_model.keras'")

# Save regression model
regression_model.save('housing_model.keras')
print("✓ Regression model saved as 'housing_model.keras'")

print("\n💡 .keras format includes:")
print("  - Model architecture")
print("  - Model weights")
print("  - Optimizer state")
print("  - Compilation settings")

In [ ]:
# Load model
loaded_model = keras.models.load_model('mnist_model.keras')

print("✓ Model loaded successfully!")
print("\nModel summary:")
loaded_model.summary()

In [ ]:
# Verify loaded model works
test_loss, test_acc = loaded_model.evaluate(X_test_norm, y_test_cat, verbose=0)

print(f"Loaded model accuracy: {test_acc:.4f}")
print("\n✓ Loaded model performs identically to original!")

### Alternative: Save Only Weights

In [ ]:
# Save only weights (smaller file size)
mnist_model.save_weights('mnist_weights.weights.h5')
print("✓ Weights saved as 'mnist_weights.weights.h5'")

print("\n💡 To load weights:")
print("  1. Recreate the model architecture")
print("  2. Load weights: model.load_weights('mnist_weights.weights.h5')")
print("  3. Compile the model (if needed for training)")

In [ ]:
# Example: Load weights into a new model
new_model = keras.Sequential([
    layers.Dense(256, input_shape=(784,)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

# Load weights
new_model.load_weights('mnist_weights.weights.h5')

# Compile (required for training or evaluation)
new_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Test
test_loss, test_acc = new_model.evaluate(X_test_norm, y_test_cat, verbose=0)
print(f"\nModel with loaded weights accuracy: {test_acc:.4f}")

---
## Section 12: Best Practices & Tips

Let's cover important concepts for building better models.

### Overfitting vs Underfitting

**Underfitting** (high bias):
- Model is too simple
- Poor performance on both training and test data
- Solution: Make model more complex, train longer

**Overfitting** (high variance):
- Model is too complex
- Great on training data, poor on test data
- Solution: Regularization (Dropout, L2), more data, early stopping

**Signs of overfitting:**
- Training accuracy >> Validation accuracy
- Validation loss increases while training loss decreases

### Using Callbacks

Callbacks are functions that run during training. Common ones:

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Early Stopping: Stop training when validation loss stops improving
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Wait 5 epochs with no improvement
    restore_best_weights=True,
    verbose=1
)

# Model Checkpoint: Save best model during training
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Reduce Learning Rate: Lower LR when learning plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Multiply LR by 0.5
    patience=3,
    min_lr=1e-7,
    verbose=1
)

print("✓ Callbacks defined")
print("\nTo use them:")
print("  model.fit(..., callbacks=[early_stop, checkpoint, reduce_lr])")

In [ ]:
# Example: Train with callbacks
example_model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(8,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

example_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train with callbacks
history_cb = example_model.fit(
    X_train_h, y_train_h,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint],
    verbose=0
)

print(f"\nTraining stopped at epoch {len(history_cb.history['loss'])}")
print("Best model saved automatically!")

### Learning Rate Tuning

Learning rate is one of the most important hyperparameters:

- **Too high**: Training is unstable, loss oscillates
- **Too low**: Training is very slow
- **Just right**: Smooth, steady improvement

**Good starting values:**
- Adam: 0.001 (default)
- SGD: 0.01 to 0.1

**Tips:**
- Start with default
- If training is slow, try higher LR
- If loss is unstable, try lower LR
- Use ReduceLROnPlateau callback

### Common Errors and Solutions

**1. Shape Mismatch**
```
Error: expected input_shape=(784,), got (28, 28)
Solution: Flatten your data or adjust input_shape
```

**2. NaN Loss**
```
Cause: Learning rate too high, data not normalized
Solution: Lower LR, normalize data, check for inf/nan in data
```

**3. Not Learning (loss not decreasing)**
```
Causes: LR too low, data not normalized, wrong loss function
Solutions: Increase LR, normalize data, verify loss function
```

**4. Overfitting**
```
Solutions: Add Dropout, get more data, reduce model complexity,
           add L2 regularization, early stopping
```

### Quick Reference

**Model Building:**
```python
model = keras.Sequential([
    layers.Dense(units, activation='relu', input_shape=(n_features,)),
    layers.Dropout(0.3),
    layers.Dense(n_classes, activation='softmax')  # or 'sigmoid' or None
])
```

**Compilation:**
```python
model.compile(
    optimizer='adam',  # or SGD, RMSprop
    loss='categorical_crossentropy',  # or 'mse', 'binary_crossentropy'
    metrics=['accuracy']  # or 'mae'
)
```

**Training:**
```python
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[...]
)
```

**Evaluation:**
```python
loss, metric = model.evaluate(X_test, y_test)
predictions = model.predict(X_new)
```

---
## Congratulations! 🎉

You've completed the Keras ANN tutorial! You now know how to:

✅ Build neural networks using Sequential and Functional APIs  
✅ Preprocess data for neural networks  
✅ Choose appropriate activations, losses, and optimizers  
✅ Train models with validation  
✅ Evaluate model performance  
✅ Apply regularization techniques  
✅ Handle classification and regression tasks  
✅ Save and load models  
✅ Use callbacks and best practices  

### Next Steps

1. Experiment with different architectures
2. Try other datasets
3. Learn about Convolutional Neural Networks (CNNs) for images
4. Explore Recurrent Neural Networks (RNNs) for sequences
5. Check out the PyTorch tutorial to compare frameworks

### Resources

- [Keras Documentation](https://keras.io/)
- [TensorFlow Tutorials](https://www.tensorflow.org/tutorials)
- [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning)

Keep learning and building! 🚀